In [1]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool
from pydantic import Field
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from crewai.tools import BaseTool

In [2]:
from crewai import LLM
llm = LLM(
    model="ollama/llama3:70b",
    base_url="http://localhost:11434"
)


In [3]:
class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: DuckDuckGoSearchAPIWrapper = Field(default_factory=DuckDuckGoSearchAPIWrapper)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"

In [4]:
researcher_agent = Agent(
    role= "{topic} Senior Data Researcher",
    goal="""Uncover cutting-edge developments in {topic}""",
    backstory="""You're a seasoned researcher with a knack for uncovering the latest
    developments in {topic}. Known for your ability to find the most relevant
    information and present it in a clear and concise manner.""",
    verbose=True,
    llm=llm,
    tools =[SearchTool()]

)

In [5]:
reporting_analyst = Agent(
    role="{topic} Reporting Analyst",
    goal="Create detailed reports based on {topic} data analysis and research findings",
    backstory= """You're a meticulous analyst with a keen eye for detail. You're known for
    your ability to turn complex data into clear and concise reports, making
    it easy for others to understand and act on the information you provide.""",
    llm= llm,
    verbose= True
)

In [6]:
research_task = Task(
    description="""
        Conduct a thorough research about AI Agents.
        Make sure you find any interesting and relevant information given
        the current year is 2024.
    """,
    expected_output="""
        A list with 10 bullet points of the most relevant information about AI Agents
    """,
    agent=researcher_agent
)

reporting_task = Task(
    description="""
        Review the context you got and expand each topic into a full section for a report.
        Make sure the report is detailed and contains any and all relevant information.
    """,
    expected_output="""
        A fully fledge reports with the mains topics, each with a full section of information.
        Formatted as markdown without '```'
    """,
    agent=reporting_analyst,
    output_file="report.md"
)

In [7]:
crew = Crew(
    agents=[researcher_agent, reporting_analyst],
    task=[research_task,reporting_task],
    verbose= True,
    process= Process.sequential
)

In [ ]:
inputs= {"topic": "AI Agents"}

out=crew.kickoff(inputs=inputs)